In [1]:
import pandas as pd
import numpy as np

years = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [24]:
crosswalk_df = pd.read_csv(r"C:\Users\david\Documents\Green House Gas Data\ghg_to_powerplant.csv")
crosswalk_df.head()

,GHGRP Facility ID,FACILITY NAME,GHGRP - City,GHGRP - State,ORIS CODE,ORIS CODE 2,ORIS CODE 3,ORIS CODE 4,ORIS CODE 5,GHGRP - Power Plant Sector,...,Reported Subparts RY11,Reported Subparts RY12,Reported Subparts RY13,Reported Subparts RY14,Reported Subparts RY15,Reported Subparts RY16,Reported Subparts RY17,Reported Subparts RY18,Reported Subparts RY19,Reported Subparts RY20
0,1000001,PSE Ferndale Generating Station,FERNDALE,WA,54537,NaN,NaN,NaN,NaN,Yes,...,C,D,D,D,D,D,D,D,D,D
1,1000019,WESTERN SUGAR COMPANY,SCOTTSBLUFF,NE,57937,NaN,NaN,NaN,NaN,NaN,...,"C,S","C,S","C,S","C,S",C,C,"C,S","C,S","C,S","C,S"
2,1000031,Northwest Regional Landfill,Surprise,AZ,57403,NaN,NaN,NaN,NaN,NaN,...,"C,HH","C,HH","C,HH","C,HH","C,HH","C,HH","C,HH","C,HH","C,HH","C,HH"
3,1000033,San Gabriel Cogeneration Facility,POMONA,CA,50300,NaN,NaN,NaN,NaN,Yes,...,C,C,C,C,C,C,C,C,C,No submission
4,1000034,ADM SOYBEAN PROCESSING - DES MOINES,DES MOINES,IA,10861,NaN,NaN,NaN,NaN,NaN,...,C,C,C,C,C,C,C,C,C,C


In [25]:
# Change the "No Match" records to 0 and remove them
crosswalk_df.loc[crosswalk_df["ORIS CODE"] == "No Match", "ORIS CODE"] = 0

crosswalk_df['ORIS CODE'] = crosswalk_df['ORIS CODE'].astype('int64')
crosswalk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2150 entries, 0 to 2149
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   GHGRP Facility ID           2150 non-null   int64  
 1   FACILITY NAME               2150 non-null   object 
 2   GHGRP - City                2150 non-null   object 
 3   GHGRP - State               2150 non-null   object 
 4   ORIS CODE                   2150 non-null   int64  
 5   ORIS CODE 2                 14 non-null     float64
 6   ORIS CODE 3                 2 non-null      float64
 7   ORIS CODE 4                 1 non-null      float64
 8   ORIS CODE 5                 1 non-null      float64
 9   GHGRP - Power Plant Sector  1775 non-null   object 
 10  Reported Subparts RY10      2150 non-null   object 
 11  Reported Subparts RY11      2150 non-null   object 
 12  Reported Subparts RY12      2150 non-null   object 
 13  Reported Subparts RY13      2150 

In [81]:
# Get all the dataframes into a dictionary
powerplant_dfs = {'powerplant_data': []}

# Loop through the 10 years of data and create a dataframe for each year and append it to a list
for y in years:
    data = pd.read_excel(r"C:\Users\david\Documents\Green House Gas Data\f923_{}\EIA923_Schedules_2_3_4_5_M_12_{}_Final_Revision.xlsx".format(y, y), sheet_name='Page 1 Generation and Fuel Data', skiprows=[0,1,2,3,4])
    data['year'] = int(y)
    
    powerplant_dfs['powerplant_data'].append(data)

In [92]:
# Merge the data frames into one dataframe per category
df_powerplant = powerplant_dfs['powerplant_data'][0]
df_powerplant = df_powerplant.iloc[:, [0, 3, 4, 5, 6, 7, 8, 10, 12, 14, 15, 18, 91, 92, 93, 94, 95, 97]]
df_powerplant.columns = [c.replace("\n", " ") for c in df_powerplant.columns]

# Loop through the lists of each type of data frame and concat them together
count = 0

for df_item in powerplant_dfs['powerplant_data']:
    df_item.columns = [c.replace("\n", " ") for c in df_item.columns]
    
    if count == 0:
        count += 1
        continue
    else:
        df_powerplant = pd.concat([df_powerplant, df_item.iloc[:, [0, 3, 4, 5, 6, 7, 8, 10, 12, 14, 15, 18, 91, 92, 93, 94, 95, 97]]])

    count += 1

In [93]:
df_powerplant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132722 entries, 0 to 15795
Data columns (total 18 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Plant Id                            132722 non-null  int64  
 1   Plant Name                          132714 non-null  object 
 2   Operator Name                       132714 non-null  object 
 3   Operator Id                         132722 non-null  object 
 4   Plant State                         132714 non-null  object 
 5   Census Region                       132722 non-null  object 
 6   NERC Region                         128741 non-null  object 
 7   NAICS Code                          132722 non-null  int64  
 8   Sector Name                         132722 non-null  object 
 9   Reported Fuel Type Code             132722 non-null  object 
 10  AER Fuel Type Code                  132722 non-null  object 
 11  Physical Unit Label        

In [86]:
# Get all the dataframes into a dictionary
ghg_dfs = {'ghg_data': []}

# Loop through the 10 years of data and create a dataframe for each year and append it to a list
for y in years:
    data = pd.read_excel(r"C:\Users\david\Documents\Green House Gas Data\2022_data_summary_spreadsheets\ghgp_data_{}.xlsx".format(y), sheet_name='Direct Emitters', skiprows=[0,1,2])
    data['year'] = int(y)
    
    ghg_dfs['ghg_data'].append(data)

In [100]:
# Merge the data frames into one dataframe per category
df_ghg = ghg_dfs['ghg_data'][0]

# Loop through the lists of each type of data frame and concat them together
for df_item in ghg_dfs['ghg_data']:
    if count == 0:
        count += 1
        continue
    else:
        df_ghg = pd.concat([df_ghg, df_item])

    count += 1

In [101]:
df_ghg.head()

,Facility Id,FRS Id,Facility Name,City,State,Zip Code,Address,County,Latitude,Longitude,...,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills,Is some CO2 collected on-site and used to manufacture other products and therefore not emitted from the affected manufacturing process unit(s)? (as reported under Subpart G or S),"Is some CO2 reported as emissions from the affected manufacturing process unit(s) under Subpart AA, G or P collected and transferred off-site or injected (as reported under Subpart PP)?",Does the facility employ continuous emissions monitoring?,year
0,1004377,1.100438e+11,121 REGIONAL DISPOSAL FACILITY,MELISSA,TX,75454,3820 SAM RAYBURN HIGHWAY,COLLIN COUNTY,33.298570,-96.535860,...,NaN,NaN,204000.0,NaN,NaN,NaN,N,N,N,2012
1,1010040,1.100701e+11,15-18565/15-18662,Hazard,KY,40701,4200 S. Hwy 15,PERRY COUNTY,37.219099,-83.156046,...,348450.75,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
2,1010085,1.100555e+11,15-19015,Hazard,KY,41701,1845 S. KY HWY 15,PERRY,37.236617,-83.181260,...,41513.00,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
3,1000112,1.100198e+11,23rd and 3rd,BROOKLYN,NY,11232,730 3rd Avenue,Kings,40.663000,-74.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
4,1006394,NaN,29-6 #2 Central Delivery Point,Blanco,NM,87412,NaN,Rio Arriba,36.745200,-107.445500,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012


In [102]:
df_ghg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75272 entries, 0 to 6520
Data columns (total 67 columns):
 #   Column                                                                                                                                                                                     Non-Null Count  Dtype  
---  ------                                                                                                                                                                                     --------------  -----  
 0   Facility Id                                                                                                                                                                                75272 non-null  int64  
 1   FRS Id                                                                                                                                                                                     74576 non-null  float64
 2   Facility Name          

In [103]:
# Get Power Plants from WA
df_wa_powerplant = df_powerplant[(df_powerplant['Plant State'] == 'WA') & (df_powerplant['NAICS Code'] == 22)]
df_wa_powerplant.head()

,Plant Id,Plant Name,Operator Name,Operator Id,Plant State,Census Region,NERC Region,NAICS Code,Sector Name,Reported Fuel Type Code,AER Fuel Type Code,Physical Unit Label,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours),year
94,99,Frederickson,Puget Sound Energy Inc,15500,WA,PACC,WECC,22,Electric Utility,DFO,DFO,barrels,707,707,4128,4128,242.229,2012
95,99,Frederickson,Puget Sound Energy Inc,15500,WA,PACC,WECC,22,Electric Utility,NG,NG,mcf,555428,555428,573203,573203,33633.771,2012
350,371,Columbia Generating Station,Energy Northwest,20160,WA,PACC,WECC,22,Electric Utility,NUC,NUC,NaN,0,0,97804137,97804137,9333709.000,2012
557,550,Kettle Falls Generating Station,Avista Corp,20169,WA,PACC,WECC,22,Electric Utility,NG,NG,mcf,9906,9906,10086,10086,860.000,2012
558,550,Kettle Falls Generating Station,Avista Corp,20169,WA,PACC,WECC,22,Electric Utility,NG,NG,mcf,3616,3616,3686,3686,226.892,2012


In [104]:
# Get GHG Records for WA Industries
df_wa_ghg = df_ghg[df_ghg['State'] == 'WA']
df_wa_ghg.head()

,Facility Id,FRS Id,Facility Name,City,State,Zip Code,Address,County,Latitude,Longitude,...,Underground Coal Mines,Zinc Production,Municipal Landfills,Industrial Wastewater Treatment,Manufacture of Electric Transmission and Distribution Equipment,Industrial Waste Landfills,Is some CO2 collected on-site and used to manufacture other products and therefore not emitted from the affected manufacturing process unit(s)? (as reported under Subpart G or S),"Is some CO2 reported as emissions from the affected manufacturing process unit(s) under Subpart AA, G or P collected and transferred off-site or injected (as reported under Subpart PP)?",Does the facility employ continuous emissions monitoring?,year
98,1004236,1.100070e+11,AGRIUM US INC - KFO,KENNEWICK,WA,99337,227515 E BOWLES ROAD,BENTON,46.165957,-119.017218,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
142,1003562,1.100409e+11,ALCOA INTALCO WORKS,FERNDALE,WA,98248,4050 MOUNTAIN VIEW RD.,WHATCOM COUNTY,48.845500,-122.705500,...,NaN,NaN,NaN,NaN,NaN,11784.0,N,N,N,2012
143,1000635,1.100005e+11,ALCOA WENATCHEE LLC,MALAGA,WA,98828,6200 MALAGA HIGHWAY,CHELAN COUNTY,47.356700,-120.127300,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
200,1004746,1.100005e+11,ANALOG DEVICES,CAMAS,WA,98607,4200 NW PACIFIC RIM BLVD,CLARK COUNTY,45.598844,-122.452320,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012
316,1002537,1.100009e+11,ASH GROVE CEMENT COMPANY-SEATTLE,SEATTLE,WA,98134,3801 EAST MARGINAL WAY SOUTH,KING COUNTY,47.567990,-122.342550,...,NaN,NaN,NaN,NaN,NaN,NaN,N,N,N,2012


In [109]:
df_both = crosswalk_df.merge(df_wa_ghg, how='right', left_on = 'GHGRP Facility ID', right_on ='Facility Id')
df_both = df_both.merge(df_wa_powerplant, how='inner', left_on = ['ORIS CODE', 'year'], right_on = ['Plant Id', 'year'])
#df_main = df_main.merge(df_wa_ghg, how='outer', left_on = 'GHGRP Facility ID', right_on ='Facility Id')

df_both.head()

,GHGRP Facility ID,FACILITY NAME,GHGRP - City,GHGRP - State,ORIS CODE,ORIS CODE 2,ORIS CODE 3,ORIS CODE 4,ORIS CODE 5,GHGRP - Power Plant Sector,...,NAICS Code,Sector Name,Reported Fuel Type Code,AER Fuel Type Code,Physical Unit Label,Total Fuel Consumption Quantity,Electric Fuel Consumption Quantity,Total Fuel Consumption MMBtu,Elec Fuel Consumption MMBtu,Net Generation (Megawatthours)
0,1007310.0,Centralia,CENTRALIA,WA,3845.0,NaN,NaN,NaN,NaN,Yes,...,22,NAICS-22 Non-Cogen,NG,NG,mcf,0,0,0,0,0.000
1,1007310.0,Centralia,CENTRALIA,WA,3845.0,NaN,NaN,NaN,NaN,Yes,...,22,NAICS-22 Non-Cogen,NG,NG,mcf,0,0,0,0,0.000
2,1007310.0,Centralia,CENTRALIA,WA,3845.0,NaN,NaN,NaN,NaN,Yes,...,22,NAICS-22 Non-Cogen,DFO,DFO,barrels,13282,13282,79692,79692,7294.378
3,1007310.0,Centralia,CENTRALIA,WA,3845.0,NaN,NaN,NaN,NaN,Yes,...,22,NAICS-22 Non-Cogen,SUB,COL,short tons,2502192,2502192,40559028,40559028,3728435.600
4,1007310.0,Centralia,CENTRALIA,WA,3845.0,NaN,NaN,NaN,NaN,Yes,...,22,NAICS-22 Non-Cogen,NG,NG,mcf,0,0,0,0,0.000


In [111]:
df_both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445 entries, 0 to 444
Columns: 105 entries, GHGRP Facility ID to Net Generation (Megawatthours)
dtypes: float64(60), int64(10), object(35)
memory usage: 368.5+ KB


In [110]:
df_both.to_csv(r"C:\Users\david\Documents\Green House Gas Data\main_data.csv")

In [105]:
df_wa_ghg.to_csv(r"C:\Users\david\Documents\Green House Gas Data\ghg_data.csv")

In [85]:
df_wa_powerplant.to_csv(r"C:\Users\david\Documents\Green House Gas Data\powerplant_data.csv")